In [76]:
from dotenv import load_dotenv
load_dotenv()

True

In [77]:
import pandas as pd
import requests
import requests_cache
from bs4 import BeautifulSoup

In [78]:
# requests_cache.install_cache('cache')

In [79]:
%run ./utils/save_result.ipynb

In [80]:
# load originals from API
originals_json = requests.get("https://www.apple.com/uk/tv-pr/originals.originals.json").json()

originals = pd.DataFrame.from_dict(originals_json["results"]["originals"])

originals = originals[originals["showType"] == "films"]

originals

,acaTitle,arialabel,genreFilters,image,showType,title,url,genres,releaseDateDisplayString,releaseDateISOString
0,Apple Originals - Argylle,Argylle - Films,[films],{'name': 'Apple_TV_Argylle_key_art_main_16_9.j...,films,Argylle,/uk/tv-pr/originals/argylle/,NaN,NaN,NaN
1,Apple Originals - Killers of the Flower Moon,Killers of the Flower Moon - Films,[films],{'name': 'Apple_TV_Killers_of_the_Flower_Moon_...,films,Killers of the Flower Moon,/uk/tv-pr/originals/killers-of-the-flower-moon/,NaN,NaN,NaN
2,Apple Originals - Louis Armstrong’s Black & Blues,Louis Armstrong’s Black & Blues - Films,[documentary],{'name': 'Apple_TV_Louis_Armstrongs_Black_Blue...,films,Louis Armstrong’s Black & Blues,/uk/tv-pr/originals/louis-armstrongs-black-blues/,[documentary],NaN,NaN
3,Apple Originals - STILL: A Michael J. Fox Movie,STILL: A Michael J. Fox Movie - Films,[films],"{'name': 'ATV_Still_key_art_main_16_9.jpg', 'v...",films,STILL: A Michael J. Fox Movie,/uk/tv-pr/originals/still-a-michael-j-fox-movie/,[documentary],NaN,NaN
5,Apple Originals - Tetris,Tetris - Films,[drama],{'name': 'Apple_TV_Tetris_key_art_main_16_9.jp...,films,Tetris,/uk/tv-pr/originals/tetris/,[drama],NaN,NaN
14,Apple Originals - Stephen Curry: Underrated,Stephen Curry: Underrated - Films,NaN,{'name': 'Apple_TV_Stephen_Curry_Underrated_ke...,films,Stephen Curry: Underrated,/uk/tv-pr/originals/stephen-curry-underrated/,[documentary],2023,2023-02-28T00:00:00.000Z
18,Apple Originals - Sharper,Sharper - Films,[drama],"{'name': 'ATV_Sharper_key_art_main_16_9.jpg', ...",films,Sharper,/uk/tv-pr/originals/sharper/,[drama],"February 17, 2023",2023-02-17T00:00:00.000Z
27,"Apple Originals - The Boy, the Mole, the Fox a...","The Boy, the Mole, the Fox and the Horse - Films",[kids & family],{'name': 'ATV_The_Boy_The_Mole_The_Fox_The_Hor...,films,"The Boy, the Mole, the Fox and the Horse",/uk/tv-pr/originals/the-boy-the-mole-the-fox-a...,[kids & family],"December 25, 2022",2022-12-25T00:00:00.000Z
30,Apple Originals - Emancipation,Emancipation - Films,[drama],{'name': 'Apple_TV_Emancipation_key_art_main_1...,films,Emancipation,/uk/tv-pr/originals/emancipation/,[drama],"December 9, 2022",2022-12-09T00:00:00.000Z
33,Apple Originals - Spirited,Spirited - Films,"[holiday, musical, films]",{'name': 'Apple_TV_Spirited_key_art_main_16_9....,films,Spirited,/uk/tv-pr/originals/spirited/,"[holiday, musical]","November 18, 2022",2022-11-18T00:00:00.000Z


In [81]:
# filter out films without a release date
originals.dropna(inplace=True, subset=["releaseDateDisplayString"])

originals.iloc[0]

acaTitle                          Apple Originals - Stephen Curry: Underrated
arialabel                                   Stephen Curry: Underrated - Films
genreFilters                                                              NaN
image                       {'name': 'Apple_TV_Stephen_Curry_Underrated_ke...
showType                                                                films
title                                               Stephen Curry: Underrated
url                             /uk/tv-pr/originals/stephen-curry-underrated/
genres                                                          [documentary]
releaseDateDisplayString                                                 2023
releaseDateISOString                                 2023-02-28T00:00:00.000Z
Name: 14, dtype: object

In [82]:
def parse_page(url: str) -> dict:
    page = requests.get(url)
    parsed_page = BeautifulSoup(page.text)

    # get title
    title = parsed_page.select("h1.description-title")[0].text

    # get year
    description_items = parsed_page.select(".description-item")
    year = None

    for item in description_items:
        if 'Release Date' in item.text:
            year = item.select(".description-fields-content")[0].text

            if ',' in year:
                year = year.split(', ')[1]
    
    

    # find director
    crew = parsed_page.select("li.cast-crew-tile")
    director = ''

    for member in crew:
        if 'Director' in member.text:
            director = member.select(".cast-crew-name-bottom")[0].text

    return {
        'title': title,
        'year': year,
        'director': director
    }

In [83]:
# convert dataframe to exportable list
films = []

for _, film in originals.iterrows():
    try:
        parsed_page = parse_page("https://www.apple.com" + film["url"])

        films.append(parsed_page)
    except Exception:
        print("error parsing: " + "https://www.apple.com" + film["url"])

films[0]

{'title': 'Stephen Curry: Underrated',
 'year': '2023',
 'director': 'Peter Nicks'}

In [84]:
save("Apple TV+ Original Films", "All Apple TV+ original feature films", "https://www.apple.com/uk/tv-pr/originals/films/", films) # type: ignore

https://film-collections.simse.dev/apple-tv-original-films.json
